<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>

In [4]:
!conda install beautifulsoup4
!conda install lxml
!conda install html5lib
!conda install requests

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lxml-4.4.2                 |   py36hefd8a0e_0         1.6 MB

The following packages will be UPDATED:

    lxml: 4.3.1-py36hefd8a0e_0 --> 4.4.2-py36hefd8a0e_0


lxml-4.4.2           | 1.6 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - requests


The following packages will be downloaded:

    package                    |            build
    --------

In [44]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

In [45]:
#Webscraping information
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

table = soup.find('table')

In [46]:
# Empty List for loading data from website
post_codes = []

for tr in table.find_all('tr')[2:]:
    post_codes.append({
            'Postcode': tr.find_all('td')[0].text,
            'Borough': tr.find_all('td')[1].text,
            'Neighborhood': re.sub('\n$', '', tr.find_all('td')[2].text)
            })
    
post_codes = pd.DataFrame(post_codes)
post_codes.head()

,Borough,Neighborhood,Postcode
0,Not assigned,Not assigned,M2A
1,North York,Parkwoods,M3A
2,North York,Victoria Village,M4A
3,Downtown Toronto,Harbourfront,M5A
4,North York,Lawrence Heights,M6A
